#### basic model eval and compare - use static pretrained global embeddings + length 
* 

In [1]:
# from bio_embeddings.embed import ProtTransBertBFDEmbedder
import pandas as pd
import numpy as np
import random

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing  import MinMaxScaler,StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier   
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedGroupKFold,GroupKFold
from sklearn.compose import make_column_selector as selector
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
# from tqdm.autonotebook import tqdm
import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
NUMERIC_ONLY = False#True # use only numbers, inore raw sequence
FAST_RUN = False#True

In [3]:
def eval_model(clf,X_train,y_train,groups,numCV=3,X_test=None,y_test=None,return_preds=True):
    train_cv_preds = cross_val_predict(estimator=clf, X=X_train, y=y_train,cv=GroupKFold(n_splits=numCV),
                                       groups=groups,method="predict_proba",n_jobs=-2)
    print("Train res")
    print(classification_report(y_train,train_cv_preds[:,1]>=.5))
    print("AUC",roc_auc_score(y_train,train_cv_preds[:,1]).round(4))
    
    if X_test is not None:
        clf.fit(X_train,y_train)
        y_test_pred = clf.predict_proba(X_test)
        print("TEST res")
        print(classification_report(y_test,y_test_pred[:,1]>=.5))
        print("AUC",roc_auc_score(y_test,y_test_pred[:,1]).round(4))
    
    if return_preds:
        return train_cv_preds

In [4]:
X_train = pd.read_csv("hum_vir_swp-globalEmbed-train.csv.gz",nrows= 2000 if FAST_RUN else 1e9)
display(X_train.iloc[:,-10:].groupby(["virus"]).select_dtypes(["number"]).agg(["mean","max","min"]).round(2))
y_train = X_train.pop("virus")
train_groups = X_train["Cluster name"]

keep_feature_cols_list = list(X_train.select_dtypes(["number","bool"]).columns)
if not NUMERIC_ONLY:
    keep_feature_cols_list = keep_feature_cols_list + ["Sequence"]
    
X_train = X_train.filter(keep_feature_cols_list,axis=1)
print(X_train.shape)
print(y_train.mean().round(3))
display(X_train)

['Entry', 'Sequence', 'Cluster name'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.


global_embed_1021             global_embed_1022              \
                   mean   max   min              mean   max   min   
virus                                                               
0                 -0.01  0.20 -0.16              0.02  0.23 -0.11   
1                  0.01  0.19 -0.14              0.03  0.21 -0.12   

      global_embed_1023              Length                Mass                \
                   mean   max   min    mean   max min      mean     max   min   
virus                                                                           
0                  0.02  0.21 -0.10  473.16  1533  11  52763.51  175639  1142   
1                 -0.01  0.14 -0.16  380.31  1520  11  42814.59  173372  1271   

        av_mw                 
         mean     max    min  
virus                         
0      111.55  138.24  81.38  
1      113.10  129.07  88.14

(20340, 1028)
0.279


,global_embed_0,global_embed_1,global_embed_2,global_embed_3,global_embed_4,global_embed_5,global_embed_6,global_embed_7,global_embed_8,global_embed_9,...,global_embed_1018,global_embed_1019,global_embed_1020,global_embed_1021,global_embed_1022,global_embed_1023,Length,Mass,av_mw,Sequence
0,0.040560,0.012460,0.04860,0.006645,-0.006474,0.027070,0.019240,-0.10620,-0.005110,0.01721,...,0.021000,0.002610,-0.004677,0.019590,-0.02592,-0.026840,102,11835,116.03,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...
1,0.007630,-0.004948,0.02852,-0.001821,-0.039600,0.066800,-0.020630,-0.16850,0.003940,-0.02774,...,0.041660,0.047820,-0.034120,0.008250,-0.00011,0.052370,362,42605,117.69,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...
2,0.037300,-0.061070,0.08730,0.005596,-0.008150,0.000575,0.011734,-0.16220,-0.013660,-0.02573,...,0.033630,-0.093140,0.093300,0.026840,0.08120,0.024750,97,10795,111.29,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...
3,-0.002218,0.002700,0.07270,-0.022480,0.001690,0.040280,0.001864,-0.17580,0.098200,0.03928,...,-0.032100,0.000459,0.019100,0.055500,0.11730,-0.012960,186,22438,120.63,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...
4,0.069760,0.098600,0.04890,0.004288,0.022430,0.062700,-0.037500,-0.05383,-0.027130,-0.02348,...,0.039180,-0.004436,-0.029570,0.034200,-0.01112,-0.009920,1360,154943,113.93,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20335,-0.041050,-0.032000,0.03632,-0.013990,-0.042300,0.070900,-0.043600,-0.06573,0.044250,0.03620,...,-0.006527,0.036900,0.000668,-0.010840,-0.01273,0.018340,578,65320,113.01,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...
20336,0.016450,-0.008040,0.04852,0.029430,-0.040300,0.040100,-0.041870,-0.11350,0.047270,0.03058,...,0.001120,-0.027710,-0.014870,-0.000851,0.06420,0.044980,348,39041,112.19,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...
20337,-0.072300,0.009674,0.08984,0.002928,-0.052500,0.059480,-0.031600,-0.07390,0.024300,0.03186,...,0.008230,0.033750,-0.027820,0.004436,0.06400,0.001059,421,48090,114.23,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...
20338,0.009220,-0.034030,0.01591,-0.005898,0.019420,0.008280,-0.054140,-0.07000,0.005203,-0.04065,...,-0.036100,-0.056500,-0.011086,-0.031430,0.05466,0.043950,376,40467,107.62,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...


In [5]:
X = X_train

In [6]:
# print(X_train.iloc[:,-10:].round(2).head().to_dict())

In [7]:
print("Test")
X_test = pd.read_csv("hum_vir_swp-globalEmbed-test.csv.gz")#,nrows= 2500 if FAST_RUN else 1e9)
X_test.drop(columns=["Cluster name"],errors="ignore",inplace=True)
y_test = X_test.pop("virus")

X_test = X_test.filter(keep_feature_cols_list,axis=1)

print(X_test.shape)
print(y_test.mean().round(3))

Test
(4777, 1028)
0.215


In [8]:
X_test

,global_embed_0,global_embed_1,global_embed_2,global_embed_3,global_embed_4,global_embed_5,global_embed_6,global_embed_7,global_embed_8,global_embed_9,...,global_embed_1018,global_embed_1019,global_embed_1020,global_embed_1021,global_embed_1022,global_embed_1023,Length,Mass,av_mw,Sequence
0,0.025620,-0.002438,0.045720,0.04990,-0.003360,0.001394,-0.055600,-0.08990,0.032170,-0.015594,...,-0.018170,-0.019180,-0.007690,0.015720,0.005882,-0.01941,102,11082,108.65,MTTATPLGDTTFFSLNMTTRGEDFLYKSSGAIVAAVVVVVIIIFTV...
1,0.048920,0.053920,0.037050,0.02147,-0.003021,0.049130,-0.021260,-0.08130,0.018370,-0.018070,...,0.061000,-0.004280,-0.029430,0.002352,-0.036800,0.03096,907,102362,112.86,MESAIAEGGASRFSASSGGGGSRGAPQHYPKTAGNSEFLGKTPGQN...
2,0.038100,0.071840,0.037840,0.02646,-0.009040,0.037170,-0.031280,-0.09094,0.030550,-0.015300,...,-0.014190,-0.046360,0.005936,-0.009346,-0.012470,-0.03450,329,34562,105.05,MGTAAAAAAAAAAAAAGEGARSPSPAAVSLGLGVAVVSSLVNGSTF...
3,-0.015030,-0.016920,0.010445,0.01985,-0.016340,0.039060,-0.031070,-0.05377,0.024690,0.009270,...,0.047820,0.048770,0.029710,-0.018880,0.059140,0.00568,460,53637,116.60,MFTIKLLLFIVPLVISSRIDQDNSSFDSLSPEPKSRFAMLDDVKIL...
4,-0.000349,0.021680,0.151700,0.02565,-0.020340,0.090100,0.021700,-0.14670,-0.029890,0.134600,...,-0.034180,-0.040900,-0.085140,0.024300,0.023480,0.03925,347,39037,112.50,MPRGQKSKLRAREKRRKAREETQGLKVAHATAAEKEECPSSSPVLG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,-0.050000,-0.029130,0.075600,0.01599,-0.022140,0.042660,0.026350,-0.07434,-0.000616,0.006010,...,-0.041930,0.026470,0.006435,0.059500,-0.001407,0.10913,164,18696,114.00,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...
4773,0.008865,-0.047000,0.015396,0.04404,-0.038450,0.085270,-0.061100,-0.06018,0.085200,0.066100,...,-0.001677,-0.011765,-0.010510,0.019680,0.049000,0.05966,1167,131616,112.78,MKVNRETKRLYVGGLSQDISEADLQNQFSRFGEVSDVEIITRKDDQ...
4774,0.042570,0.050170,0.009445,0.02354,0.011020,0.076050,-0.026640,-0.06350,0.017720,0.000740,...,0.036040,-0.008484,-0.009030,0.015730,0.020370,0.04540,567,64260,113.33,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...
4775,-0.017070,0.005505,0.044560,-0.01119,-0.005690,0.047180,0.032840,-0.07556,0.018000,-0.002062,...,-0.003290,-0.053900,0.005430,-0.048580,-0.021910,0.03400,283,31623,111.74,MGASVSRGRAARVPAPEPEPEEALDLSQLPPELLLVVLSHVPPRTL...


In [9]:
if NUMERIC_ONLY:
    clf = HistGradientBoostingClassifier()
    clf_lin = Pipeline(steps=[
        ('preprocessor', StandardScaler()),
         ('classifier',LogisticRegressionCV(Cs=7,n_jobs=-1,max_iter=600))
else:  
    # Identify numerical and text features
    num_features = [i for i in X.columns if i != 'Sequence']
    text_features = ['Sequence']

    # For numerical features, we use a simple identity function
#     num_transformer = FunctionTransformer(func=lambda x: x, validate=True)
    num_transformer = StandardScaler()

    # For the text feature, we use TfidfVectorizer with char-level n-grams
    text_transformer = Pipeline(steps=[
    ('tfidf',TfidfVectorizer(analyzer='char', ngram_range=(2, 3),
                                       min_df=4, max_df=0.98,sublinear_tf=True,
                                       lowercase=False,max_features=7000)),
    ('chi2', SelectKBest(chi2, k=300))  # Select 20 best features
])
    

    # Build ColumnTransformer for preprocessing step of clf
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_features),
        ('text', text_transformer, text_features[0])
    ],remainder="passthrough")

    # Create clf
    clf_lin = Pipeline(steps=[
        ('preprocessor', preprocessor),
#         ('classifier', RandomForestClassifier(n_jobs=-1,)#GradientBoostingClassifier() ## gbc is slow 
         # min_samples_leaf=2,
         # hist gbc - doesn't suppot sparse? 
         ('classifier',
          LogisticRegressionCV(Cs=7,n_jobs=-2,max_iter=600)
         )
        
    ])

    clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
#         ('classifier', RandomForestClassifier(n_jobs=-1,)#GradientBoostingClassifier() ## gbc is slow 
     # min_samples_leaf=2,
     # hist gbc - doesn't suppot sparse? 
     ('classifier',HistGradientBoostingClassifier()
     )

])
# # Fit the model
# clf.fit(X_train, y_train)
# print("Trained")


In [10]:
# %%time
# ##slow? 

# ## # Get the feature names extracted by TF-IDF
# ## feature_names = num_features + preprocessor.transformers_[1][1].get_feature_names_out()
# #ALT: (with FS)# Get the feature names extracted by TF-IDF and chi2
# feature_names = num_features + [f'text_{i}' for i in range(text_transformer.named_steps['chi2'].k)]

# # Create SHAP explainer
# explainer = shap.TreeExplainer(clf.named_steps['classifier'])
# print("Explainer")
# # Calculate SHAP values
# shap_values = explainer.shap_values(preprocessor.transform(X_test.head(55)))

# # Summarize the SHAP values in a pandas DataFrame for better visualization
# shap_df = pd.DataFrame(shap_values[0], columns=feature_names)

# # Print SHAP values
# # print(shap_df)

# shap_values = explainer(preprocessor.transform(X_test.head(55)),y_test.head(55))

# # # visualize the first prediction's explanation
# # shap.plots.waterfall(shap_values[0])

# # shap.plots.beeswarm(shap_values)
# ## hack for random forest:
# shap.plots.beeswarm(shap_values[:,:,1]) ## laks column/feature names..

In [11]:
%%time
train_cv_preds = eval_model(clf=clf,X_train=X_train,y_train=y_train,groups=train_groups,
           numCV=4,X_test=X_test,y_test=y_test)

Train res
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     14668
           1       0.95      0.90      0.92      5672

    accuracy                           0.96     20340
   macro avg       0.96      0.94      0.95     20340
weighted avg       0.96      0.96      0.96     20340

AUC 0.991823997856843
TEST res
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3750
           1       0.93      0.96      0.94      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.96      4777
weighted avg       0.98      0.98      0.98      4777

AUC 0.9957582603050958
CPU times: user 4min 34s, sys: 25.1 s, total: 4min 59s
Wall time: 1min 23s


In [12]:
%%time
train_cv_preds_linear =eval_model(clf=clf_lin,X_train=X_train,y_train=y_train,groups=train_groups,
           numCV=4,X_test=X_test,y_test=y_test,return_preds=True)

/Users/oferd2/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/oferd2/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

Train res
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     14668
           1       0.96      0.92      0.94      5672

    accuracy                           0.97     20340
   macro avg       0.96      0.95      0.96     20340
weighted avg       0.97      0.97      0.97     20340

AUC 0.9932795930271244


/Users/oferd2/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/oferd2/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

TEST res
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3750
           1       0.94      0.95      0.94      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.96      4777
weighted avg       0.98      0.98      0.98      4777

AUC 0.9956476468679001
CPU times: user 43.7 s, sys: 3.88 s, total: 47.6 s
Wall time: 14min 32s


* ensembl preds

In [19]:
train_cv_preds_linear.shape

(20340, 2)

In [24]:
## similar or slightly better perf - negligible difference
joint_preds = (train_cv_preds_linear[:,1] + train_cv_preds[:,1])/2
print(joint_preds.shape)
print(classification_report(y_train,joint_preds>=.5))
print("AUC",roc_auc_score(y_train,joint_preds).round(4))

## overwrite old preds
train_cv_preds = joint_preds

(20340,)
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     14668
           1       0.96      0.92      0.94      5672

    accuracy                           0.97     20340
   macro avg       0.96      0.95      0.96     20340
weighted avg       0.97      0.97      0.97     20340

AUC 0.994


### predictions error analysis

In [26]:
df_preds = pd.read_csv("hum_vir_swp-globalEmbed-train.csv.gz").drop(columns=["Sequence"])
df_preds = df_preds.filter(list(df_preds.select_dtypes(["O"]).columns) + ["Length","virus"],axis=1)
df_preds["model_pred_proba"] = train_cv_preds.round(3)
# [:,1]
df_preds["model_pred"] = (train_cv_preds>=.5).astype(int)
# df_preds["mismatch"] = (df_preds["model_pred"]!=df_preds["virus"])
## keep only mismatch cases
df_preds = df_preds.loc[df_preds["model_pred"]!=df_preds["virus"]].sort_values(["model_pred_proba"])
df_preds

,Entry,Cluster name,Length,virus,model_pred_proba,model_pred
1962,Q9IZK2,"Cluster: Beta-1,3-galactosyl-O-glycosyl-glycop...",440,1,0.000,0
2459,P11364,Cluster: Viral T-cell receptor beta chain-like...,321,1,0.000,0
186,Q2HRC6,Cluster: Putative Dihydrofolate reductase,210,1,0.000,0
4968,P56260,Cluster: Transforming protein Qin,387,1,0.000,0
17645,P17863,Cluster: Transforming protein Ski,437,1,0.001,0
...,...,...,...,...,...,...
12543,P09430,Cluster: Spermatid nuclear transition protein 1,55,0,0.996,1
4732,Q14507,Cluster: Epididymal secretory protein E3-alpha,147,0,0.996,1
1805,H0YL14,Cluster: Transmembrane protein 250,139,0,0.997,1
16216,P56279,Cluster: T-cell leukemia/lymphoma protein 1A,114,0,0.997,1


In [27]:
display(df_preds[df_preds["model_pred"]==0])

,Entry,Cluster name,Length,virus,model_pred_proba,model_pred
1962,Q9IZK2,"Cluster: Beta-1,3-galactosyl-O-glycosyl-glycop...",440,1,0.000,0
2459,P11364,Cluster: Viral T-cell receptor beta chain-like...,321,1,0.000,0
186,Q2HRC6,Cluster: Putative Dihydrofolate reductase,210,1,0.000,0
4968,P56260,Cluster: Transforming protein Qin,387,1,0.000,0
17645,P17863,Cluster: Transforming protein Ski,437,1,0.001,0
...,...,...,...,...,...,...
2122,Q8BDD8,Cluster: Protein E7,98,1,0.498,0
14112,P03235,Cluster: Uncharacterized protein EC-RF4,289,1,0.498,0
19373,P19553,Cluster: Protein Tat,101,1,0.499,0
15454,Q9YUD0,Cluster: Uncharacterized protein ORF7,86,1,0.499,0


In [28]:
display(df_preds[df_preds["model_pred"]==0].iloc[0:20,0:3])

,Entry,Cluster name,Length
1962,Q9IZK2,"Cluster: Beta-1,3-galactosyl-O-glycosyl-glycop...",440
2459,P11364,Cluster: Viral T-cell receptor beta chain-like...,321
186,Q2HRC6,Cluster: Putative Dihydrofolate reductase,210
4968,P56260,Cluster: Transforming protein Qin,387
17645,P17863,Cluster: Transforming protein Ski,437
8384,Q49P94,Cluster: Golgi anti-apoptotic protein,237
12889,P03400,Cluster: 18 kDa protein,163
10817,P01114,Cluster: Transforming protein p29,248
12397,Q00996,Cluster: Surface glycoprotein CD59 homolog,121
3212,P68678,Cluster: Viral interleukin-10 homolog,179


In [29]:
display(df_preds[df_preds["model_pred"]==1].sort_values(["model_pred_proba"],ascending=False))

,Entry,Cluster name,Length,virus,model_pred_proba,model_pred
4793,Q5VT40,Cluster: Protein FAM78B,261,0,0.999,1
16216,P56279,Cluster: T-cell leukemia/lymphoma protein 1A,114,0,0.997,1
1805,H0YL14,Cluster: Transmembrane protein 250,139,0,0.997,1
4732,Q14507,Cluster: Epididymal secretory protein E3-alpha,147,0,0.996,1
12543,P09430,Cluster: Spermatid nuclear transition protein 1,55,0,0.996,1
...,...,...,...,...,...,...
18105,P59022,Cluster: Down syndrome critical region protein 10,87,0,0.508,1
19104,Q8N456,Cluster: Leucine-rich repeat-containing protei...,261,0,0.508,1
5071,Q537H7,Cluster: Spermatogenesis-associated protein 45,98,0,0.508,1
5718,P04554,Cluster: Protamine-2,102,0,0.505,1


In [30]:
display(df_preds[df_preds["model_pred"]==1].sort_values(["model_pred_proba"],ascending=False).iloc[0:20,0:3])

,Entry,Cluster name,Length
4793,Q5VT40,Cluster: Protein FAM78B,261
16216,P56279,Cluster: T-cell leukemia/lymphoma protein 1A,114
1805,H0YL14,Cluster: Transmembrane protein 250,139
4732,Q14507,Cluster: Epididymal secretory protein E3-alpha,147
12543,P09430,Cluster: Spermatid nuclear transition protein 1,55
11056,Q9H496,"Cluster: Torsin-1A-interacting protein 2, isof...",131
17559,Q96BQ5,Cluster: Coiled-coil domain-containing protein...,260
8446,Q8N8F7,Cluster: Leucine-rich single-pass membrane pro...,131
17106,Q9UI54,Cluster: Putative uncharacterized protein PRO0628,55
8955,Q9H2U6,Cluster: Putative uncharacterized protein enco...,94
